In [1]:
import os
from dotenv import load_dotenv
from typing import Any, Optional

from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator
from langchain.smith import EvaluationConfig, run_on_dataset

# --- Setup ---
# Load environment variables and initialize the LangSmith client
load_dotenv()
client = Client()

# --- Part 1: Run Evaluation with a Built-in Evaluator ---
print("--- Running Evaluation with Built-in Evaluator ---")

# Define a simple Q&A chain to test
qa_prompt = PromptTemplate.from_template("You are a helpful assistant. Answer the following question: {question}")
qa_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=qa_prompt)

# Use the EvaluationConfig object to configure the evaluation
eval_config_builtin = EvaluationConfig(
    evaluators=["qa"]
)

# Run the evaluation on the Q&A dataset
# Note: Ensure you created a dataset named "LangSmith Docs Q&A" in Lesson 1
run_on_dataset(
    client=client,
    dataset_name="LangSmith Docs Q&A",
    llm_or_chain_factory=qa_chain,
    evaluation=eval_config_builtin,
    project_name="qa-evaluation-final",
)
print("Finished running evaluation with built-in 'qa' evaluator.")


# --- Part 2: My Custom Tweak - A Custom Evaluator for Conciseness ---
print("\n--- Running Evaluation with Custom Evaluator (My Tweak) ---")

# Define our custom evaluator function
def is_concise(run, example) -> dict:
    """
    Custom evaluator that checks if the output summary is less than 75 characters.
    """
    llm_output = run.outputs.get("text")
    if llm_output is None:
        return {"key": "is_concise", "score": 0}
    
    is_concise_bool = len(llm_output) < 75
    return {"key": "is_concise", "score": int(is_concise_bool)}

# Wrap our custom function in LangChainStringEvaluator
custom_evaluator = LangChainStringEvaluator([is_concise])

# Define a simple summarization chain
summary_prompt = PromptTemplate.from_template("Summarize the following article in one sentence: {article}")
summary_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=summary_prompt)

# Use EvaluationConfig for the custom evaluator as well
eval_config_custom = EvaluationConfig(
    evaluators=[custom_evaluator]
)

# Run evaluation on our summarization dataset
# Note: Ensure you created a dataset named "Article Summaries" in Lesson 1
run_on_dataset(
    client=client,
    dataset_name="Article Summaries",
    llm_or_chain_factory=summary_chain,
    evaluation=eval_config_custom,
    project_name="summary-evaluation-final",
)
print("Finished running evaluation with custom 'is_concise' evaluator.")

ImportError: cannot import name 'EvaluationConfig' from 'langchain.smith' (c:\Users\dell\Documents\CODES\LangSmith\ls-academy\Lib\site-packages\langchain\smith\__init__.py)